In [5]:
import geopandas as gpd
import pandas as pd
import folium
from scipy.spatial import Voronoi
from shapely.geometry import Polygon
import json

# Read tower locations from CSV file into a DataFrame
tower_df1 = pd.read_csv("C:\\Users\\HP\\OneDrive\\Desktop\\Internship Kochi\\EKM_Timesort\\abc1.csv")
tower_df1 = tower_df1.head(19999)
tower_df1['Tower_id'] = tower_df1['Lati'].map(str) + '-' + tower_df1['Longi'].map(str)
tower_df=pd.DataFrame(tower_df1.groupby('Tower_id').first(),index=None)

In [6]:
tower_df.reset_index(inplace=True)

In [7]:
# Create a GeoDataFrame for tower locations
tower_gdf = gpd.GeoDataFrame(tower_df, geometry=gpd.points_from_xy(tower_df.Longi, tower_df.Lati))

# Calculate Voronoi diagram
points = tower_gdf.geometry.apply(lambda geom: (geom.x, geom.y)).tolist()
vor = Voronoi(points)

# Extract the actual polygons from the Voronoi diagram and convert them to Shapely geometries
voronoi_polygons = [Polygon(vor.vertices[region]) for region in vor.regions if -1 not in region]

# Create a GeoDataFrame for Voronoi polygons
voronoi_gdf = gpd.GeoDataFrame({'geometry': voronoi_polygons}, crs=tower_gdf.crs)


In [8]:
# Assuming you have a GeoDataFrame representing the city boundary
# Load or create the 'city_boundary_gdf' here
city_boundary_gdf = gpd.read_file('C:\\Users\\HP\\OneDrive\\Desktop\\Internship Kochi\\Ernk_shape\\ernak\\ernak.shp')

In [9]:
# Clip the Thiessen polygons to the city boundary
thiessen_within_city = gpd.overlay(voronoi_gdf, city_boundary_gdf, how='intersection')

# Create a Folium map centered on the mean coordinates of the city boundary
m = folium.Map(location=[city_boundary_gdf["geometry"].centroid.y, city_boundary_gdf["geometry"].centroid.x], zoom_start=12)

# Plot the Thiessen polygons within the city boundary on the Folium map
for _, row in thiessen_within_city.iterrows():
    folium.GeoJson(data=row["geometry"].__geo_interface__).add_to(m)

# Plot the tower locations as markers on the map
for _, row in tower_gdf.iterrows():
    folium.Marker([row["Lati"], row["Longi"]], popup=row["Tower_id"]).add_to(m)
    
# gdf = gpd.GeoDataFrame.from_features(json.loads(geojson_data['features']))
# gdf.to_file("C:\\Users\\HP\\OneDrive\\Desktop\\Internship Kochi\\Kochi_Metro\\shape_file_poly.shp")

# Display the interactive map
m


C:\Users\HP\AppData\Local\Temp\ipykernel_13904\1080653101.py:2: UserWarning: CRS mismatch between the CRS of left geometries and the CRS of right geometries.
Use `to_crs()` to reproject one of the input geometries to match the CRS of the other.

Left CRS: None
Right CRS: EPSG:4326

  thiessen_within_city = gpd.overlay(voronoi_gdf, city_boundary_gdf, how='intersection')
C:\Users\HP\AppData\Local\Temp\ipykernel_13904\1080653101.py:5: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  m = folium.Map(location=[city_boundary_gdf["geometry"].centroid.y, city_boundary_gdf["geometry"].centroid.x], zoom_start=12)
